In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

print(train_data.shape, test_data.shape) #42000, 785 / 28000, 784
print(train_data.head(1)) ##label, pixel 0~783
print(test_data.head(1))  #pixel 0~783

In [ ]:
import numpy as np

train_x = train_data.drop(['label'], axis = 1)
train_y = train_data['label']

#28*28로 변환해야 함.
train_x = train_x.values.reshape(-1, 28, 28, 1) #흑백이니까 1
test_data = test_data.values.reshape(-1, 28, 28, 1)

max = np.max(train_x)

#Normalize
train_x = train_x / max
test_data = test_data / max

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback

class cb(Callback) :
    def on_epoch_end(self, epoch, logs = {}) :
        if(logs.get('loss') < 0.001) :
            self.model.stop_training = True

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(28, 28, 1), padding='same'))
model.add(MaxPool2D((2,2))) #14
model.add(Conv2D(128, (3,3), padding='same'))
model.add(MaxPool2D((2,2))) #7
model.add(Conv2D(128, (3,3), padding='same'))
model.add(MaxPool2D((2,2))) #3
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

n_epoch = 100
with tf.device('/gpu:0') :
    model.fit(train_x, train_y, verbose = 1,epochs = n_epoch,
              callbacks = [cb()], batch_size = 1000)

In [ ]:
import random
import matplotlib.pyplot as plt
%matplotlib inline

predict = model.predict(test_data)
predict=np.argmax(predict, axis=1)

In [ ]:
#20 개만 뽑아서 시각화
r = random.sample(range(0, len(test_data)), 20)

fig = plt.figure(figsize = (15, 15))

for i in range(20) :
    subplot = fig.add_subplot(5, 4, i+1)
    subplot.set_title(predict[r[i]])
    subplot.imshow(test_data[r[i]], cmap=plt.cm.gray_r)

plt.show()

In [ ]:
#ImageId, Label로 저장해야 함.
predict = pd.Series(predict, name = 'Label')
ids = [i for i in range(1, len(predict)+1)]

save_data = pd.concat([pd.Series(ids, name='ImageId'),
                      predict], axis = 1)

print(save_data.head(3))
print(save_data.tail(3))

save_data.to_csv('result.csv', index = False)